In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [2]:
with open("Conversa do WhatsApp com New school.txt", 'r', encoding='utf-8')as file:
    linhas = file.readlines()

In [3]:
dados = []
for linha in linhas:
    match = re.match(r'(\d{2}/\d{2}/\d{4} \d{2}:\d{2}) - (.+?): (.+)', linha)
    if match:
        data_hora = match.group(1)
        remetente = match.group(2)
        mensagem = match.group(3)
        dados.append({'data_hora': data_hora, 'remetente': remetente, 'mensagem': mensagem})

In [ ]:
df = pd.DataFrame(dados)
df.head

In [ ]:
#1. Resumo das conversas:
resumo = df['remetente'].value_counts().reset_index()
resumo.columns = ['remetente', 'totalMsg']
resumo = resumo.sort_values(by='totalMsg', ascending=False)

print(resumo)

In [ ]:
#2. Histórico do remetente:
todosRementes = df['remetente'].unique()
print("\nRemetentes disponíveis:")
for i, remetente in enumerate(todosRementes, start=1):
    print(f"{i}. {remetente}")

escolha = int(input("Digite o número correspondente ao remetente que deseja filtrar: ")) - 1

if 0 <= escolha < len(todosRementes):
    remetenteEspecifico = todosRementes[escolha]

    msgFiltrada = df[df['remetente'].str.strip() == remetenteEspecifico.strip()]

    print(f"\nMensagens de {remetenteEspecifico}:")
    print(msgFiltrada[['data_hora', 'mensagem']])

else:
    print("Escolha inválida. Tente novamente.")

In [ ]:
#3. Gráfico do histórico do remetente:
df['data_hora'] = pd.to_datetime(df['data_hora'], format='%d/%m/%Y %H:%M')
df['data'] = df['data_hora'].dt.date

contagemMsg = df.groupby(['data', 'remetente']).size().reset_index(name='totalMsg')

plt.figure(figsize=(12, 6))
for remetente in contagemMsg['remetente'].unique():
    dadosRemetente = contagemMsg[contagemMsg['remetente'] == remetente]
    plt.bar(dadosRemetente['data'], dadosRemetente['totalMsg'], label=remetente)

plt.title('Quantidade de Conversas por Dia por Remetente')
plt.xlabel('Data')
plt.ylabel('Total de Mensagens')
plt.xticks(rotation=45)
plt.legend(title='Remetente')
plt.tight_layout()
plt.show()

In [ ]:
#4. Gráfico de pizza:
totalMsg = df['remetente'].value_counts()

plt.figure(figsize=(8, 8))
plt.pie(totalMsg, labels=totalMsg.index, autopct='%1.1f%%', startangle=90)
plt.title('Percentual de Mensagens por Remetente')
plt.show()

In [ ]:
#5. Gráfico de linhas:
plt.figure(figsize=(12, 6))
for remetente in contagemMsg['remetente'].unique():
    dadosRemetente = contagemMsg[contagemMsg['remetente'] == remetente]
    plt.plot(dadosRemetente['data'], dadosRemetente['totalMsg'], marker='o', label=remetente)

plt.title('Quantidade de Mensagens ao Longo do Tempo por Remetente')
plt.xlabel('Data')
plt.ylabel('Total de Mensagens')
plt.xticks(rotation=45)
plt.legend(title='Remetente')
plt.tight_layout()
plt.grid()
plt.show()